In [1]:
import pandas as pd
from datetime import datetime, date, timedelta
from pathlib import Path
from tqdm import tqdm
import shutil
import os, stat
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', 500)

<div style="background-color:#B2EBF2; font-size:12pt; border: 1px solid #35BDD0; border-radius: 5px; padding:10px">
Перед запуском <b>копируем исходники в папку</b> указанную ниже
</div>

In [3]:
# Путь к папке с исходными файлами

folder_path = Path('z:/Проектный офис/Протоколы/')

In [4]:
# Создание пустого списка для хранения DataFrame’ов

frames = []

In [5]:
# Получаем список файлов для объединения

files_list = [file for file in folder_path.glob('DAT8651_*.xlsx')]

In [6]:
files_list

[WindowsPath('z:/Проектный офис/Протоколы/DAT8651_паллиатив_2024-10-01.xlsx'),
 WindowsPath('z:/Проектный офис/Протоколы/DAT8651_паллиатив_2024-10-02.xlsx'),
 WindowsPath('z:/Проектный офис/Протоколы/DAT8651_паллиатив_2024-10-03.xlsx'),
 WindowsPath('z:/Проектный офис/Протоколы/DAT8651_паллиатив_2024-10-04.xlsx'),
 WindowsPath('z:/Проектный офис/Протоколы/DAT8651_паллиатив_2024-10-05.xlsx'),
 WindowsPath('z:/Проектный офис/Протоколы/DAT8651_паллиатив_2024-10-06.xlsx'),
 WindowsPath('z:/Проектный офис/Протоколы/DAT8651_паллиатив_2024-10-07.xlsx'),
 WindowsPath('z:/Проектный офис/Протоколы/DAT8651_паллиатив_2024-10-08.xlsx'),
 WindowsPath('z:/Проектный офис/Протоколы/DAT8651_паллиатив_2024-10-09.xlsx'),
 WindowsPath('z:/Проектный офис/Протоколы/DAT8651_паллиатив_2024-10-10.xlsx'),
 WindowsPath('z:/Проектный офис/Протоколы/DAT8651_паллиатив_2024-10-11.xlsx'),
 WindowsPath('z:/Проектный офис/Протоколы/DAT8651_паллиатив_2024-10-12.xlsx'),
 WindowsPath('z:/Проектный офис/Протоколы/DAT8651_па

In [7]:
# Объединение всех файлов .xlsx в один большой файл

for file_path in tqdm(files_list, desc='Объединение файлов', unit='файл'):
    df = pd.read_excel(file_path, sheet_name='Sheet', dtype='object')
    # df['файл'] = file_path
    frames.append(df)

Объединение файлов: 100%|██████████| 26/26 [00:30<00:00,  1.19s/файл]


In [8]:
# Объединение всех DataFrame’ов из списка

merged_frame = pd.concat(frames, ignore_index=True)

In [9]:
df_emias = merged_frame.copy()

In [10]:
df_emias['Дата протокола'] = pd.to_datetime(df_emias['Дата протокола'], errors ='coerce').dt.date

In [11]:
df_emias.shape

(110797, 34)

In [12]:
df_emias.drop_duplicates(inplace=True)

In [13]:
df_emias.shape

(34531, 34)

In [14]:
df_emias.loc[  ~df_emias['Описание помощи в амбулаторных условиях'].isna() , 
                 'list_тип_визита'] = df_emias['Описание помощи в амбулаторных условиях'].str.split(';')

In [15]:
df_emias.loc[  ~df_emias['Дата следующего планового визита'].isna() , 
                 'list_дата_план_визита'] = df_emias['Дата следующего планового визита'].str.split(';')

In [16]:
df_emias.loc[ ( df_emias['list_тип_визита'].str[0] == 'визит врача'), 'план_дата_визит_врач' ] = df_emias['list_дата_план_визита'].str[0]
df_emias.loc[ ( df_emias['list_тип_визита'].str[1] == ' визит врача'), 'план_дата_визит_врач' ] = df_emias['list_дата_план_визита'].str[1]
df_emias.loc[ ( df_emias['list_тип_визита'].str[2] == ' визит врача'), 'план_дата_визит_врач' ] = df_emias['list_дата_план_визита'].str[2]
df_emias.loc[ ( df_emias['list_тип_визита'].str[3] == ' визит врача'), 'план_дата_визит_врач' ] = df_emias['list_дата_план_визита'].str[3]

In [17]:
df_emias['план_дата_визит_врач'] = pd.to_datetime(df_emias['план_дата_визит_врач'].str.strip()).dt.date
df_emias['ID пациента'] = df_emias['ID пациента'].astype('str')

In [18]:
df_emias = df_emias[[
                    'ID пациента',
                    'ОМС пациента',
                    'Дата протокола',
                    'ФИО врача',
                    'Бригада врача',
                    'ОВПП',
                    'Тип протокола',
                    'Диагноз пациента',
                    'Показания к паллиативной помощи',
                    'Категория пациента',
                    'Оказание паллиативной помощи',
                    'Состояние пациента',
                    'Болевой синдром',
                    'Оценка болевого синдрома на момент осмотра в баллах (ВАШ)',
                    'Оценка болевого синдрома на момент осмотра в баллах (НОШ)',
                    'Оценка болевого синдрома на момент осмотра в баллах (painaid)',
                    'Трахеостома',
                    'Респираторная поддержка',
                    'Мочеиспускание',
                    'Асцит',
                    'Установлено устройство',
                    'Дисфагия',
                    'Дисфагия. Степень',
                    'PPI',
                    'PPS',
                    'По шкале Морсе в баллах',
                    'По шкале Нортон в баллах',
                    'Описание помощи в амбулаторных условиях',
                    'Патронажный план',
                    'Частота посещений',
                    'Дата следующего планового визита',
                    'Дата следующего планового визита. С целью',
                    'Дата следующего планового визита. Причина',
                    'Манипуляции/Процедуры. Проведено',
                    # 'list_тип_визита',
                    # 'list_дата_план_визита',
                    'план_дата_визит_врач'
                     ]]

In [19]:
file_save_path = 'z:/Проектный офис/Протоколы/Все/Протокол ЕМИАС все визиты_' + datetime.now().strftime("%Y%m%d") + '.xlsx'

with pd.ExcelWriter(file_save_path,
                      engine="xlsxwriter",
                      date_format='DD.MM.YYYY',
                      datetime_format='DD.MM.YYYY HH:MM:SS') as writer:
    df_emias.to_excel(writer, index = False)

os.chmod(file_save_path, stat.S_IREAD)

In [20]:
# Оставляем только крайние визиты

df_emias = df_emias.sort_values(by=['ID пациента', 'Дата протокола'], ascending=[True, False]).drop_duplicates(subset=['ID пациента'])

In [21]:
file_save_path = 'z:/Проектный офис/Протоколы/Крайние визиты/Протокол ЕМИАС крайние визиты_' + datetime.now().strftime("%Y%m%d") + '.xlsx'

with pd.ExcelWriter(file_save_path,
                      engine="xlsxwriter",
                      date_format='DD.MM.YYYY',
                      datetime_format='DD.MM.YYYY HH:MM:SS') as writer:
    df_emias.to_excel(writer, index = False)

os.chmod(file_save_path, stat.S_IREAD)